In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

import torch
import torch.utils.data as Data
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torch.autograd import Variable
from torchsummary import summary

import os
import random
import datetime

# Setting

In [3]:
#  predition TI of leading time at 24 hours
pre_seq = 4
batch_size = 128
epochs = 128
min_val_loss = 100
model_name = 'SAF_Net.pkl'
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [4]:
train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/intensity/TI_Prediction-master/data/CMA_train_'+str(pre_seq*6)+'h.csv', header=None)
test= pd.read_csv('/content/drive/My Drive/Colab Notebooks/intensity/TI_Prediction-master/data/CMA_test_'+str(pre_seq*6)+'h.csv', header=None)

In [5]:
train.shape, test.shape

((8406, 101), (2747, 101))

In [6]:
CLIPER_feature =  pd.concat((train, test), axis=0)
CLIPER_feature.reset_index(drop=True, inplace=True)

In [7]:
X_wide_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()

X_wide = X_wide_scaler.fit_transform(CLIPER_feature.iloc[:, 5:])
X_wide_train = X_wide[0: train.shape[0], :]

y = y_scaler.fit_transform(CLIPER_feature.loc[:, 3].values.reshape(-1, 1))
y_train = y[0: train.shape[0], :]

In [8]:

ahead_times = [0]
pressures = [1, 2, 3,5, 7, 10,20, 30, 50,70, 100, 125,150, 175, 200,225,
             250, 300,350, 400, 450,500, 550, 600,650, 700, 750,775,
             800, 825,850, 875, 900,925, 950, 975,1000]



sequential_reanalysis_u_list = []
reanalysis_u_test_dict = {}
X_deep_u_scaler_dict = {}

sequential_reanalysis_v_list = []
reanalysis_v_test_dict = {}
X_deep_v_scaler_dict = {}

sequential_reanalysis_t_list = []
reanalysis_t_test_dict = {}
X_deep_t_scaler_dict = {}


sequential_reanalysis_r_list = []
reanalysis_r_test_dict = {}
X_deep_r_scaler_dict = {}

In [9]:
reanalysis_type = 'v'
for ahead_time in ahead_times:
    reanalysis_list = []
    for pressure in pressures:
        folder = None
        if ahead_time == 0:
            folder = reanalysis_type
        else:
            folder = reanalysis_type + '_' + str(ahead_time*6)
        all_reanalysis_csv = pd.read_csv('/content/drive/My Drive/Colab Notebooks/typhoon/data/ERA_Interim/'+folder+'/reanalysis_'+reanalysis_type+'_'+str(pressure)+'.csv', header=None)
      

        train_reanalysis = all_reanalysis_csv[all_reanalysis_csv[0].isin(train[0].unique())]
        test_reanalysis = all_reanalysis_csv[all_reanalysis_csv[0].isin(test[0].unique())]
        reanalysis_v_test_dict[reanalysis_type+str(pressure)+str(ahead_time)] = test_reanalysis # 保存test 用于后面测试

        reanalysis =  pd.concat((train_reanalysis, test_reanalysis), axis=0)
        reanalysis.reset_index(drop=True, inplace=True)

        scaler_name = reanalysis_type +str(pressure) + str(ahead_time)
        X_deep_v_scaler_dict[scaler_name] = MinMaxScaler()
        
        # 5:end is the 31*31 v component wind speed
        X_deep = X_deep_v_scaler_dict[scaler_name].fit_transform(reanalysis.loc[:, 5:])
        
        # (batch, type, channel, height, widht, time) here type is v
        X_deep_final = X_deep[0: train.shape[0], :].reshape(-1, 1, 1, 3, 3, 1)
        reanalysis_list.append(X_deep_final)
        
    X_deep_temp = np.concatenate(reanalysis_list[:], axis=2)
    print("ahead_time:", ahead_time, X_deep_temp.shape)
    sequential_reanalysis_v_list.append(X_deep_temp)

X_deep_v_train = np.concatenate(sequential_reanalysis_v_list, axis=5)

ahead_time: 0 (8406, 1, 37, 3, 3, 1)


In [10]:
reanalysis_type = 't'
for ahead_time in ahead_times:
    reanalysis_list = []
    for pressure in pressures:
        folder = None
        if ahead_time == 0:
            folder = reanalysis_type
        else:
            folder = reanalysis_type + '_' + str(ahead_time*6)

        all_reanalysis_csv = pd.read_csv('/content/drive/My Drive/Colab Notebooks/typhoon/data/ERA_Interim/'+folder+'/reanalysis_'+reanalysis_type+'_'+str(pressure)+'.csv', header=None)
      
        train_reanalysis = all_reanalysis_csv[all_reanalysis_csv[0].isin(train[0].unique())]
        test_reanalysis = all_reanalysis_csv[all_reanalysis_csv[0].isin(test[0].unique())]
        reanalysis_t_test_dict[reanalysis_type+str(pressure)+str(ahead_time)] = test_reanalysis # 保存test 用于后面测试
        
        reanalysis =  pd.concat((train_reanalysis, test_reanalysis), axis=0)
        reanalysis.reset_index(drop=True, inplace=True)

        scaler_name = reanalysis_type +str(pressure) + str(ahead_time)
        X_deep_t_scaler_dict[scaler_name] = MinMaxScaler()
        
        # 5:end is the 31*31 u component wind speed
        X_deep = X_deep_t_scaler_dict[scaler_name].fit_transform(reanalysis.loc[:, 5:])
        
        # (batch, type, channel, height, widht, time) here type is u
        X_deep_final = X_deep[0: train.shape[0], :].reshape(-1, 1, 1, 3, 3, 1)
        reanalysis_list.append(X_deep_final)

    X_deep_temp = np.concatenate(reanalysis_list[:], axis=2)
    print("ahead_time:", ahead_time, X_deep_temp.shape)
    sequential_reanalysis_t_list.append(X_deep_temp)

X_deep_t_train = np.concatenate(sequential_reanalysis_t_list, axis=5)

ahead_time: 0 (8406, 1, 37, 3, 3, 1)


In [11]:
reanalysis_type = 'u'
for ahead_time in ahead_times:
    reanalysis_list = []
    for pressure in pressures:
        folder = None
        if ahead_time == 0:
            folder = reanalysis_type
        else:
            folder = reanalysis_type + '_' + str(ahead_time*6)

        all_reanalysis_csv = pd.read_csv('/content/drive/My Drive/Colab Notebooks/typhoon/data/ERA_Interim/'+folder+'/reanalysis_'+reanalysis_type+'_'+str(pressure)+'.csv', header=None)
      

        train_reanalysis = all_reanalysis_csv[all_reanalysis_csv[0].isin(train[0].unique())]
        test_reanalysis = all_reanalysis_csv[all_reanalysis_csv[0].isin(test[0].unique())]
        reanalysis_u_test_dict[reanalysis_type+str(pressure)+str(ahead_time)] = test_reanalysis # 保存test 用于后面测试
        
        reanalysis =  pd.concat((train_reanalysis, test_reanalysis), axis=0)
        reanalysis.reset_index(drop=True, inplace=True)

        scaler_name = reanalysis_type +str(pressure) + str(ahead_time)
        X_deep_u_scaler_dict[scaler_name] = MinMaxScaler()
        
        # 5:end is the 31*31 u component wind speed
        X_deep = X_deep_u_scaler_dict[scaler_name].fit_transform(reanalysis.loc[:, 5:])
        
        # (batch, type, channel, height, widht, time) here type is u
        X_deep_final = X_deep[0: train.shape[0], :].reshape(-1, 1, 1, 3, 3, 1)
        reanalysis_list.append(X_deep_final)

    X_deep_temp = np.concatenate(reanalysis_list[:], axis=2)
    print("ahead_time:", ahead_time, X_deep_temp.shape)
    sequential_reanalysis_u_list.append(X_deep_temp)

X_deep_u_train = np.concatenate(sequential_reanalysis_u_list, axis=5)

ahead_time: 0 (8406, 1, 37, 3, 3, 1)


In [12]:
reanalysis_type = 'r'
for ahead_time in ahead_times:
    reanalysis_list = []
    for pressure in pressures:
        folder = None
        if ahead_time == 0:
            folder = reanalysis_type
        else:
            folder = reanalysis_type + '_' + str(ahead_time*6)

        all_reanalysis_csv = pd.read_csv('/content/drive/My Drive/Colab Notebooks/typhoon/data/ERA_Interim/'+folder+'/reanalysis_'+reanalysis_type+'_'+str(pressure)+'.csv', header=None)
      

        train_reanalysis = all_reanalysis_csv[all_reanalysis_csv[0].isin(train[0].unique())]
        test_reanalysis = all_reanalysis_csv[all_reanalysis_csv[0].isin(test[0].unique())]
        reanalysis_r_test_dict[reanalysis_type+str(pressure)+str(ahead_time)] = test_reanalysis # 保存test 用于后面测试
        
        reanalysis =  pd.concat((train_reanalysis, test_reanalysis), axis=0)
        reanalysis.reset_index(drop=True, inplace=True)

        scaler_name = reanalysis_type +str(pressure) + str(ahead_time)
        X_deep_r_scaler_dict[scaler_name] = MinMaxScaler()
        
        # 5:end is the 31*31 u component wind speed
        X_deep = X_deep_r_scaler_dict[scaler_name].fit_transform(reanalysis.loc[:, 5:])
        
        # (batch, type, channel, height, widht, time) here type is u
        X_deep_final = X_deep[0: train.shape[0], :].reshape(-1, 1, 1, 3, 3, 1)
        reanalysis_list.append(X_deep_final)

    X_deep_temp = np.concatenate(reanalysis_list[:], axis=2)
    print("ahead_time:", ahead_time, X_deep_temp.shape)
    sequential_reanalysis_r_list.append(X_deep_temp)

X_deep_r_train = np.concatenate(sequential_reanalysis_r_list, axis=5)

ahead_time: 0 (8406, 1, 37, 3, 3, 1)


In [13]:

X_deep_train = np.concatenate((X_deep_u_train, X_deep_v_train,X_deep_t_train,X_deep_r_train), axis=1)

In [14]:
X_wide_train.shape, X_deep_train.shape

((8406, 96), (8406, 4, 37, 3, 3, 1))

# training set and validation set

In [15]:
class TrainLoader(Data.Dataset):
    def __init__(self, X_wide_train, X_deep_train, y_train):
        self.X_wide_train = X_wide_train
        self.X_deep_train = X_deep_train
        self.y_train = y_train
        
    def __getitem__(self, index):
        return [self.X_wide_train[index], self.X_deep_train[index]], self.y_train[index]
    
    def __len__(self):
        return len(self.X_wide_train)

In [16]:
full_train_index = [*range(0, len(X_wide_train))]

train_index, val_index, _, _, = train_test_split(full_train_index,full_train_index,test_size=0.1)

In [17]:
len(train_index), len(val_index)

(7565, 841)

In [18]:
train_dataset = torch.utils.data.DataLoader(
    TrainLoader(X_wide_train[train_index], X_deep_train[train_index], y_train[train_index]), 
                                                 batch_size=batch_size, shuffle=True)

In [19]:
val_dataset = torch.utils.data.DataLoader(
    TrainLoader(X_wide_train[val_index], X_deep_train[val_index], y_train[val_index]), 
                                                 batch_size=batch_size, shuffle=True)

# SAF-Net

In [20]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        # deep
        self.conv1 = nn.Conv2d(37, 64, kernel_size=1, padding=(1, 1))
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=1, padding=(1, 1))
        self.conv3 = nn.Conv2d(128, 256, kernel_size=1, padding=(1, 1))
        self.fc1 = nn.Linear(256 *2 *2, 128)

        self.fc2 = nn.Linear(96 + 128 * len(ahead_times), 64)
        self.fc3 = nn.Linear(64, 1)
        self.fc4=nn.Linear(128,1)

        self.num_layers=1
        self.hidden_size=128
        self.gru = nn.GRU(input_size=96, hidden_size=128,
                            num_layers=1, batch_first=True)
        self.gru2 = nn.GRU(input_size=self.hidden_size, hidden_size=self.hidden_size,
                           num_layers=self.num_layers, batch_first=True)
        self.gru3 = nn.GRU(input_size=self.hidden_size, hidden_size=self.hidden_size,
                           num_layers=self.num_layers, batch_first=True)
        self.gru4 = nn.GRU(input_size=self.hidden_size, hidden_size=self.hidden_size,
                           num_layers=self.num_layers, batch_first=True)

    def forward(self, wide, deep):
        seq_list = []
        seq_list2 = []
        seq_list3 = []
        seq_list4 = []

        x = wide.unsqueeze(1)
        h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size,device=x.device).float())
        h_1 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size,device=x.device).float())
        h_2 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size,device=x.device).float())
        h_3 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size,device=x.device).float())
        ula,_  = self.gru(x, h_0)
        ula2, _ = self.gru2(ula,h_1)
        ula3, _ = self.gru3(ula2, h_2)
        ula4, _ = self.gru4(ula3, h_3)
        h_out = ula4.view(-1, self.hidden_size)
        h_out = self.fc4(h_out)
        for i in range(len(ahead_times)):
            
            deep_u = deep[:,0,:,:,:,i]
            deep_v = deep[:,1,:,:,:,i]
            deep_t = deep[:,2,:,:,:,i]
            deep_r = deep[:,3,:,:,:,i]
            
           
            deep_u = self.pool(F.relu(self.conv1(deep_u)))
            deep_v = self.pool(F.relu(self.conv1(deep_v)))
            deep_t = self.pool(F.relu(self.conv1(deep_t)))
            deep_r = self.pool(F.relu(self.conv1(deep_r)))
           
           
            deep_u = self.pool(F.relu(self.conv2(deep_u))) 
            deep_v = self.pool(F.relu(self.conv2(deep_v)))
            deep_t = self.pool(F.relu(self.conv2(deep_t))) 
            deep_r = self.pool(F.relu(self.conv2(deep_r)))
            

           
            deep_u = self.pool(F.relu(self.conv3(deep_u)))
            deep_v = self.pool(F.relu(self.conv3(deep_v)))
            deep_t = self.pool(F.relu(self.conv3(deep_t)))
            deep_r = self.pool(F.relu(self.conv3(deep_r)))


           
            time_seq = deep_u.view(-1, 256 * 2 * 2)
            time_seq2 = deep_v.view(-1, 256 * 2 * 2)
            time_seq3 = deep_t.view(-1, 256 * 2 * 2)
            time_seq4 = deep_r.view(-1, 256 * 2 * 2)

            time_seq = self.fc1(time_seq)
            time_seq2 = self.fc1(time_seq2)
            time_seq3 = self.fc1(time_seq3)
            time_seq4 = self.fc1(time_seq4)


            seq_list.append(time_seq)
            seq_list2.append(time_seq2)
            seq_list3.append(time_seq3)
            seq_list4.append(time_seq4)


        wide = wide.view(-1, 96)
        wide_n_deep_u = torch.cat((wide, seq_list[0]),1)
        if len(ahead_times) > 1:
            for i in range(1, len(ahead_times)):
                wide_n_deep = torch.cat((wide_n_deep_u, seq_list[i]),1)
        wide_n_deep_u = F.relu(self.fc2(wide_n_deep_u))
        wide_n_deep_u = F.relu(self.fc3(wide_n_deep_u))

        wide_n_deep_v = torch.cat((wide, seq_list2[0]),1)
        if len(ahead_times) > 1:
            for i in range(1, len(ahead_times)):
                wide_n_deep_v = torch.cat((wide_n_deep_v, seq_list2[i]),1)
        wide_n_deep_v = F.relu(self.fc2(wide_n_deep_v))
        wide_n_deep_v = F.relu(self.fc3(wide_n_deep_v))


        wide_n_deep_t = torch.cat((wide, seq_list3[0]),1)
        if len(ahead_times) > 1:
            for i in range(1, len(ahead_times)):
                wide_n_deep_t = torch.cat((wide_n_deep_t, seq_list3[i]),1)
        wide_n_deep_t = F.relu(self.fc2(wide_n_deep_t))
        wide_n_deep_t = F.relu(self.fc3(wide_n_deep_t))

        wide_n_deep_r = torch.cat((wide, seq_list4[0]),1)
        if len(ahead_times) > 1:
            for i in range(1, len(ahead_times)):
                wide_n_deep_r = torch.cat((wide_n_deep_r, seq_list4[i]),1)
        wide_n_deep_r = F.relu(self.fc2(wide_n_deep_r))
        wide_n_deep_r = F.relu(self.fc3(wide_n_deep_r))

        return (wide_n_deep_u+wide_n_deep_v+wide_n_deep_t+wide_n_deep_r+h_out)/5
    


net = Net()

In [21]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net = net.to(device)

# Training the network

In [22]:
criterion = nn.L1Loss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.0007)

In [23]:
import torch
import torch.nn as nn
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(device=None))

1
Tesla T4


In [ ]:
full_train_index = [*range(0, len(X_wide_train))]

for epoch in range(epochs):  # loop over the dataset multiple times
    train_index, val_index, _, _, = train_test_split(full_train_index,full_train_index,test_size=0.1)
    train_dataset = torch.utils.data.DataLoader(
        TrainLoader(X_wide_train[train_index], X_deep_train[train_index], y_train[train_index]), 
                                                 batch_size=batch_size,)
    val_dataset = torch.utils.data.DataLoader(
        TrainLoader(X_wide_train[val_index], X_deep_train[val_index], y_train[val_index]), 
                                                 batch_size=batch_size,)
    # training
    total_train_loss = 0
    for step, (batch_x, batch_y) in enumerate(train_dataset):
        if torch.cuda.is_available():
            net.cuda()
            X_wide_train_cuda = batch_x[0].float().cuda()
            X_deep_train_cuda = batch_x[1].float().cuda()
            y_train_cuda = batch_y.cuda()
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        pred_y = net(X_wide_train_cuda, X_deep_train_cuda)
        loss = criterion(pred_y, y_train_cuda)
        total_train_loss += loss.item()
        loss.backward()
        optimizer.step()
    
    # validating
    total_val_loss = 0
    for _,(batch_val_x, batch_val_y) in enumerate(val_dataset):
        
        if torch.cuda.is_available():
            X_wide_val_cuda = batch_val_x[0].float().cuda()
            X_deep_val_cuda = batch_val_x[1].float().cuda()
            y_val_cuda = batch_val_y.cuda()
        
        pred_y = net(X_wide_val_cuda, X_deep_val_cuda)
        val_loss = criterion(pred_y, y_val_cuda)
        total_val_loss += val_loss.item()
        # print statistics
    if min_val_loss > total_val_loss:
        torch.save(net.state_dict(), model_name)
        min_val_loss = total_val_loss
    print('epochs [%d/%d] train_loss: %.5f val_loss: %.5f' % (epoch + 1, epochs, total_train_loss, total_val_loss))

print('Finished Training')

epochs [1/128] train_loss: 7.99468 val_loss: 0.60397
epochs [2/128] train_loss: 4.54784 val_loss: 0.48214
epochs [3/128] train_loss: 4.16145 val_loss: 0.51370
epochs [4/128] train_loss: 4.15420 val_loss: 0.49393
epochs [5/128] train_loss: 3.96354 val_loss: 0.47212
epochs [6/128] train_loss: 3.88372 val_loss: 0.51077
epochs [7/128] train_loss: 3.89883 val_loss: 0.46134
epochs [8/128] train_loss: 3.72554 val_loss: 0.45048
epochs [9/128] train_loss: 3.69347 val_loss: 0.41587
epochs [10/128] train_loss: 3.66802 val_loss: 0.44055
epochs [11/128] train_loss: 3.59499 val_loss: 0.44259
epochs [12/128] train_loss: 3.58375 val_loss: 0.46458
epochs [13/128] train_loss: 3.57938 val_loss: 0.43170
epochs [14/128] train_loss: 3.52385 val_loss: 0.40284
epochs [15/128] train_loss: 3.56099 val_loss: 0.44123
epochs [16/128] train_loss: 3.56128 val_loss: 0.42358
epochs [17/128] train_loss: 3.44547 val_loss: 0.42707
epochs [18/128] train_loss: 3.49681 val_loss: 0.38044
epochs [19/128] train_loss: 3.42898 v

# Testing

In [ ]:
net.load_state_dict(torch.load('/content/'+model_name))


years = test[4].unique()

test_list = []

for year in years:
    temp = test[test[4]==year]
    temp = temp.reset_index(drop=True)
    test_list.append(temp)
    
len(test_list)

In [ ]:
total=0
with torch.no_grad():
    for year, _test in zip(years, test_list):

        print(year, '年:')
        y_test = _test.loc[:,3]
        X_wide_test = X_wide_scaler.transform(_test.loc[:,5:])

        final_test_u_list = []
        for ahead_time in ahead_times:
            year_test_list = []
            for pressure in pressures:
                scaler_name = 'u' +str(pressure) + str(ahead_time)
                X_deep = reanalysis_u_test_dict[scaler_name][reanalysis_u_test_dict[scaler_name][0].isin(_test[0].unique())].loc[:,5:]
                X_deep = X_deep_u_scaler_dict[scaler_name].transform(X_deep)
                X_deep_final = X_deep.reshape(-1, 1, 1, 3, 3, 1)
                year_test_list.append(X_deep_final)
            X_deep_temp = np.concatenate(year_test_list, axis=2)
            final_test_u_list.append(X_deep_temp)
        X_deep_u_test = np.concatenate(final_test_u_list, axis=5)
        
        final_test_v_list = []
        for ahead_time in ahead_times:
            year_test_list = []
            for pressure in pressures:
                scaler_name = 'v' +str(pressure) + str(ahead_time)
                X_deep = reanalysis_v_test_dict[scaler_name][reanalysis_v_test_dict[scaler_name][0].isin(_test[0].unique())].loc[:,5:]
                X_deep = X_deep_v_scaler_dict[scaler_name].transform(X_deep)
                X_deep_final = X_deep.reshape(-1, 1, 1, 3, 3, 1)
                year_test_list.append(X_deep_final)
            X_deep_temp = np.concatenate(year_test_list, axis=2)
            final_test_v_list.append(X_deep_temp)
        X_deep_v_test = np.concatenate(final_test_v_list, axis=5)

        final_test_t_list = []
        for ahead_time in ahead_times:
            year_test_list = []
            for pressure in pressures:
                scaler_name = 't' +str(pressure) + str(ahead_time)
                X_deep = reanalysis_t_test_dict[scaler_name][reanalysis_t_test_dict[scaler_name][0].isin(_test[0].unique())].loc[:,5:]
                X_deep = X_deep_t_scaler_dict[scaler_name].transform(X_deep)
                X_deep_final = X_deep.reshape(-1, 1, 1, 3, 3, 1)
                year_test_list.append(X_deep_final)
            X_deep_temp = np.concatenate(year_test_list, axis=2)
            final_test_t_list.append(X_deep_temp)
        X_deep_t_test = np.concatenate(final_test_t_list, axis=5)

        final_test_r_list = []
        for ahead_time in ahead_times:
            year_test_list = []
            for pressure in pressures:
                scaler_name = 'r' +str(pressure) + str(ahead_time)
                X_deep = reanalysis_r_test_dict[scaler_name][reanalysis_r_test_dict[scaler_name][0].isin(_test[0].unique())].loc[:,5:]
                X_deep = X_deep_r_scaler_dict[scaler_name].transform(X_deep)
                X_deep_final = X_deep.reshape(-1, 1, 1, 3, 3, 1)
                year_test_list.append(X_deep_final)
            X_deep_temp = np.concatenate(year_test_list, axis=2)
            final_test_r_list.append(X_deep_temp)
        X_deep_r_test = np.concatenate(final_test_r_list, axis=5)

    
        X_deep_test = np.concatenate((X_deep_u_test, X_deep_v_test,X_deep_t_test,X_deep_r_test), axis=1)
        
        if torch.cuda.is_available():
            X_wide_test = Variable(torch.from_numpy(X_wide_test).float().cuda())
            X_deep_test = Variable(torch.from_numpy(X_deep_test).float().cuda())

        pred = net(X_wide_test, X_deep_test)

        pred = y_scaler.inverse_transform(pred.cpu().detach().numpy().reshape(-1,1))
        true = y_test.values.reshape(-1, 1)
        diff = np.abs(pred - true)

        print('avg wind error:', sum(diff)/len(diff))
        total=total+sum(diff)/len(diff)
print("The mean avg is::",total/len(test_list))